# CrewAI with AgentNeo Integration

This Jupyter notebook demonstrates the integration of AgentNeo, a powerful tracing and monitoring tool, with CrewAI, a framework for orchestrating role-playing AI agents. This integration allows for comprehensive analysis and debugging of AI-powered systems.

## Setup and Imports

First, let's import the necessary libraries and set up our environment.

In [ ]:
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv

from agentneo import AgentNeo, Tracer, launch_dashboard

# Load environment variables
load_dotenv("YOUR_ENV_FILE_PATH")

## Initialize AgentNeo Session and Tracer

Now, let's set up our AgentNeo session and tracer.

In [ ]:
# Create a session
neo_session = AgentNeo(session_name="crewai_example")

# Connect to a Project
neo_session.create_project(project_name="crewai_example_project")

# Create tracer
tracer = Tracer(session=neo_session)

tracer.start()

## Define Helper Functions

Let's define some helper functions for our AI tools, using AgentNeo's tracing capabilities.

In [ ]:
from langchain_community.tools import TavilySearchResults

travily_tool = tracer.wrap_langchain_tool(TavilySearchResults())

## Define Agents

Now, let's create our AI agents using CrewAI.

In [ ]:
@tracer.trace_agent("researcher")
def create_researcher():
    return Agent(
        role='Senior Research Analyst',
        goal='Uncover cutting-edge developments in AI and data science',
        backstory="""You work at a leading tech think tank.
        Your expertise lies in identifying emerging trends.
        You have a knack for dissecting complex data and presenting actionable insights.""",
        verbose=True,
        allow_delegation=False,
        tools=[travily_tool],
    )

@tracer.trace_agent("writer")
def create_writer():
    return Agent(
        role='Tech Content Strategist',
        goal='Craft compelling content on tech advancements',
        backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
        You transform complex concepts into compelling narratives.""",
        verbose=True,
        allow_delegation=True,
    )

researcher = create_researcher()
writer = create_writer()

## Define Tasks

Let's create tasks for our agents.

In [ ]:
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

## Create and Execute Crew

Now, let's create our crew and execute the tasks.

In [ ]:
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  process=Process.sequential,
  verbose=True,
)

# Get your crew to work!
result = crew.kickoff()

print(result)

tracer.stop()

## Stop Tracing and Launch Dashboard

Finally, let's stop the tracer and launch the AgentNeo dashboard to analyze the results.

In [ ]:
launch_dashboard()

This notebook demonstrates how to integrate AgentNeo's tracing capabilities with CrewAI, allowing for detailed analysis and monitoring of AI agent interactions and task execution. The AgentNeo dashboard provides valuable insights into the performance and behavior of your AI system.